# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [223]:
problem = '''Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets. 

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs).'''

## 2. Ask for parameters

In [224]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [225]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Decision Variables:

*  *w* = Acres of land allocated to wheat production.
*  *c* = Acres of land allocated to corn production.
*  *s* = Acres of land allocated to sugar beet production.

## Auxiliary Variables:

* *w_b* = Tons of wheat purchased.
* *c_b* = Tons of corn purchased.
* *w_s* = Tons of wheat sold.
* *c_s* = Tons of corn sold.
* *s_q* = Tons of sugar beets sold at quota price.
* *s_e* = Tons of sugar beets sold in excess of quota. 


# 2. Ask for objective

In [226]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [227]:
Markdown(response2.text)

Minimize:

 150*w + 230*c + 260*s  + 216*w_b + 180*c_b - 170*w_s - 150*c_s - 36*s_q - 10*s_e 


# 3. Ask for constraints

In [228]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [229]:
Markdown(response3.text)

## Constraints:

**Land availability:**

* w + c + s <= 500 

**Production and purchase balance:**

* 2.5w + w_b - w_s >= 200  
* 3c + c_b - c_s >= 240 

**Sugar beet quota:**

* s_q <= 6000

**Relationship between production, sales, and quota:**

* 20s - s_q - s_e = 0

**Non-negativity of variables:**

* w, c, s, w_b, c_b, w_s, c_s, s_q, s_e >= 0 


## 3. Generate the pyomo code

In [230]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [231]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data if available
land_availability = 500
wheat_demand = 200
corn_demand = 240
sugar_beet_quota = 6000

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.w = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land for wheat
model.c = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land for corn
model.s = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land for sugar beet

# Auxiliary Variables
model.w_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat purchased
model.c_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn purchased
model.w_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat sold
model.c_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn sold
model.s_q = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beets sold at quota price
model.s_e = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beets sold in excess of quota

# Objective Function
model.profit = pyo.Objective(
    expr=150 * model.w
    + 230 * model.c
    + 260 * model.s
    + 216 * model.w_b
    + 180 * model.c_b
    - 170 * model.w_s
    - 150 * model.c_s
    - 36 * model.s_q
    - 10 * model.s_e,
    sense=pyo.minimize,
)

# Constraints
model.land_constraint = pyo.Constraint(
    expr=model.w + model.c + model.s <= land_availability
)
model.wheat_balance = pyo.Constraint(
    expr=2.5 * model.w + model.w_b - model.w_s >= wheat_demand
)
model.corn_balance = pyo.Constraint(
    expr=3 * model.c + model.c_b - model.c_s >= corn_demand
)
model.sugar_quota = pyo.Constraint(expr=model.s_q <= sugar_beet_quota)
model.sugar_balance = pyo.Constraint(
    expr=20 * model.s - model.s_q - model.s_e == 0
)

# Solve the model
solver = pyo.SolverFactory("glpk")  # Choose a suitable solver
results = solver.solve(model)

# Print the results
print(results)
print("Profit:", pyo.value(model.profit))
print("Wheat Acres:", pyo.value(model.w))
print("Corn Acres:", pyo.value(model.c))
print("Sugar Beet Acres:", pyo.value(model.s))
```


## 4. Input problem data and try running the generated code

In [232]:
import pyomo.environ as pyo

# Sample Data - Replace with actual data if available
land_availability = 500
wheat_demand = 200
corn_demand = 240
sugar_beet_quota = 6000

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.w = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land for wheat
model.c = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land for corn
model.s = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land for sugar beet

# Auxiliary Variables
model.w_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat purchased
model.c_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn purchased
model.w_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat sold
model.c_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn sold
model.s_q = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beets sold at quota price
model.s_e = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beets sold in excess of quota

# Objective Function
model.profit = pyo.Objective(
    expr=150 * model.w
    + 230 * model.c
    + 260 * model.s
    + 216 * model.w_b
    + 180 * model.c_b
    - 170 * model.w_s
    - 150 * model.c_s
    - 36 * model.s_q
    - 10 * model.s_e,
    sense=pyo.minimize,
)

# Constraints
model.land_constraint = pyo.Constraint(
    expr=model.w + model.c + model.s <= land_availability
)
model.wheat_balance = pyo.Constraint(
    expr=2.5 * model.w + model.w_b - model.w_s >= wheat_demand
)
model.corn_balance = pyo.Constraint(
    expr=3 * model.c + model.c_b - model.c_s >= corn_demand
)
model.sugar_quota = pyo.Constraint(expr=model.s_q <= sugar_beet_quota)
model.sugar_balance = pyo.Constraint(
    expr=20 * model.s - model.s_q - model.s_e == 0
)

# Solve the model
solver = pyo.SolverFactory("glpk")  # Choose a suitable solver
results = solver.solve(model)

# Print the results
print(results)
print("Profit:", pyo.value(model.profit))
print("Wheat Acres:", pyo.value(model.w))
print("Corn Acres:", pyo.value(model.c))
print("Sugar Beet Acres:", pyo.value(model.s))


Problem: 
- Name: unknown
  Lower bound: -118600.0
  Upper bound: -118600.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 9
  Number of nonzeros: 13
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04641079902648926
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit: -118599.99999999999
Wheat Acres: 120.0
Corn Acres: 80.0
Sugar Beet Acres: 300.0


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [233]:
print(response.text)

## Decision Variables:

*  *w* = Acres of land allocated to wheat production.
*  *c* = Acres of land allocated to corn production.
*  *s* = Acres of land allocated to sugar beet production.

## Auxiliary Variables:

* *w_b* = Tons of wheat purchased.
* *c_b* = Tons of corn purchased.
* *w_s* = Tons of wheat sold.
* *c_s* = Tons of corn sold.
* *s_q* = Tons of sugar beets sold at quota price.
* *s_e* = Tons of sugar beets sold in excess of quota. 



In [234]:
print(response2.text)

Minimize:

 150*w + 230*c + 260*s  + 216*w_b + 180*c_b - 170*w_s - 150*c_s - 36*s_q - 10*s_e 



In [235]:
print(response3.text)

## Constraints:

**Land availability:**

* w + c + s <= 500 

**Production and purchase balance:**

* 2.5w + w_b - w_s >= 200  
* 3c + c_b - c_s >= 240 

**Sugar beet quota:**

* s_q <= 6000

**Relationship between production, sales, and quota:**

* 20s - s_q - s_e = 0

**Non-negativity of variables:**

* w, c, s, w_b, c_b, w_s, c_s, s_q, s_e >= 0 



In [236]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data if available
land_availability = 500
wheat_demand = 200
corn_demand = 240
sugar_beet_quota = 6000

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.w = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land for wheat
model.c = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land for corn
model.s = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land for sugar beet

# Auxiliary Variables
model.w_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat purchased
model.c_b = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn purchased
model.w_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat sold
model.c_s = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn sold
model.s_q = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beets sold at quota price
model.s_e = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beets sold in excess of quota

# Objective Function
model.profit